## 3.3 Creating a BRDF model

In [1]:
from __future__ import print_function, division
import os
import numpy as np
import itertools
import multiprocessing
import ctypes
import glob
from run_start import *
from generate_camera_file import *
from plot_polar_contour import *
%matplotlib inline

## Method
With each iteration the azimuth and zenith view angles are permeated through angles 0 - 360$^{\circ}$ in 10$^{\circ}$ steps and 5$^{\circ}$ - 60$^{\circ}$ in 5$^{\circ}$ steps respectively.  The illumination azimuth and zenith angles are set to 90$^{\circ}$ and 75$^{\circ}$ repsectively in <code><span style="background-color:lightgrey">light/brdf.light</span></code>.

This code introduces a Python module that can construct a <code><span style="background-color:lightgrey">*.camera</span></code> from a dictionary. 

In [2]:
azimuth = np.arange(0, 370, 10)
zenith = -np.arange(-60, 0, 5)
coordinates = np.asarray(list(itertools.product(azimuth, zenith)))

In [3]:
# http://stackoverflow.com/questions/5549190/is-shared-readonly-data-copied-to-different-processes-for-python-multiprocessing/5550156#5550156

shared_array_base = multiprocessing.Array(ctypes.c_double, len(coordinates) * 3)
shared_array = np.ctypeslib.as_array(shared_array_base.get_obj())
shared_array = shared_array.reshape(len(coordinates), 3)

shared_array[:, :2] = coordinates

def process_(i, def_param=shared_array):
    
    A = shared_array[i, 0]
    Z = shared_array[i, 1]
    
    # create tmeporary camera file
    camera_fname = 'camera/brdf.{}.camera'.format(i)
    results_name = 'output/brdf.{}'.format(i)
    camera(camera_fname,
           geometry_options={'geometry.azimuth':A,
                             'geometry.zenith':Z,
                             'geometry.lookAt':[.2, 0, 5.36],
                             'geometry.idealArea':1},
           results_options={'oname':results_name},
           remove=['result.image'], overwrite=True) # we don't need an image

    # run simulation
    cmd = "echo 14 {} light/brdf_light.dat | \
           start -RATm 5 -RATsensor_wavebands wb/brdf_waveband.dat \
           obj/birch_tree_scene.obj".format(camera_fname)
    error = run_start(cmd)

    # extract reflectance value
    shared_array[i, 2] = np.loadtxt(results_name + '.results.direct')[:, 1].sum()
    
    # delete files
    os.unlink(camera_fname)
    os.unlink(results_name + '.results')
    os.unlink(results_name + '.results.direct')
    os.unlink(results_name + '.results.diffuse')

if __name__ == '__main__':
    
    number_of_processes = 20
    estimated_time = (len(shared_array) / number_of_processes) / 60.
    
    print('this will run {} tasks'.format(len(shared_array)))
    print('as each run takes ~1 min (on my mac) therefore whole process should take {:.2f} hours'.format(estimated_time))

    pool = multiprocessing.Pool(processes=number_of_processes)
    pool.map(process_, [i for i in range(len(shared_array))])

this will run 444 tasks
as each run takes ~1 min (on my mac) therefore whole process should take 0.37 hours


FileNotFoundError: [Errno 2] No such file or directory: 'camera/brdf.90.camera'

In [ ]:
brdf_array = shared_array[:, 2].reshape(len(azimuth), len(zenith))

In [ ]:
plot_polar_contour(brdf_array, azimuth, zenith)